# Introdução

Esse caderno tem por objetivo a criação de um modelo básico de treinamento, utilizando o classificador [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), que faça a predição de homologação de arquivamentos de procedimentos enviados à 1A.CAM do MPF.

Esse modelo usará apenas os metadados dos procedimentos, sem fazer nenhum processamento textual.

O objetivo desse modelo é servir como um *baseline* de comparações para implementações futuras.


**Nota**: os dados desse modelo foram recuperados de procedimentos que tiveram suas deliberações realizadas após o dia 02/07/2018, data em que a nova composição tomou posse na 1A.CAM.

# Carga de dados e pré-processamento

Vamos fazer a carga dos dados e fazer um pré-processamento tradicional (remoção de atributos que não interessam, criação de variáveis categóricas etc.)

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

from sklearn import metrics

In [5]:
PATH = "../data/"
df_original = pd.read_json(f'{PATH}/1A.CAM.homologacao-arquivamento.json')

In [6]:
len(df_original)

5462

In [7]:
df_original.columns

Index(['areaAtuacao', 'classe', 'dataAutuacao', 'dataEntrada', 'homologado',
       'id', 'itemCnmp', 'municipio', 'prioritario', 'procedimento',
       'quantidadeConversoes', 'quantidadeProvidencias', 'urgente'],
      dtype='object')

In [8]:
df_original.sample(10)

,areaAtuacao,classe,dataAutuacao,dataEntrada,homologado,id,itemCnmp,municipio,prioritario,procedimento,quantidadeConversoes,quantidadeProvidencias,urgente
4199,2,2,15/03/2016,09/08/2016,1,70421190,1514,810.0,0,1.15.000.000715/2016-50,0,2,0
2965,5,3,02/07/2015,17/03/2017,1,65683120,3147,3001.0,0,1.17.001.000151/2015-90,3,8,0
665,2,3,05/07/2013,06/06/2018,1,47493892,1556,3671.0,0,1.34.012.000415/2013-33,5,34,0
378,2,3,19/11/2015,04/07/2018,1,68539308,3147,4980.0,0,1.20.002.000185/2015-89,2,2,0
4394,2,3,12/08/2013,13/06/2016,1,48067303,1564,3689.0,0,1.34.001.004840/2013-21,4,25,0
2246,5,3,23/10/2015,03/07/2017,1,68060362,930,5388.0,0,1.36.000.000972/2015-81,3,11,0
4939,2,2,09/10/2015,03/03/2016,1,67787294,1533,810.0,0,1.15.000.002514/2015-14,0,1,0
3404,2,3,15/06/2016,24/01/2017,1,72129699,1514,5459.0,0,1.16.000.002186/2016-91,3,5,0
2580,2,2,18/08/2016,04/05/2017,1,73305851,1654,5459.0,0,1.16.000.002946/2016-61,1,2,0
4765,2,3,29/07/2015,18/04/2016,1,66425158,1507,3955.0,0,1.25.000.002430/2015-26,2,7,0


In [11]:
removed_columns = ['id', 'procedimento']
df_original = df_original.drop(columns=removed_columns)
df_original.sample(10)

,areaAtuacao,classe,dataAutuacao,dataEntrada,homologado,itemCnmp,municipio,prioritario,quantidadeConversoes,quantidadeProvidencias,urgente
737,5,3,28/02/2014,30/05/2018,0,1521,810.0,0,4,16,0
5151,2,3,18/06/2013,26/01/2016,1,1520,4570.0,0,3,15,0
4778,5,2,25/01/2016,22/04/2016,1,1863,129.0,0,1,2,0
2839,2,2,07/06/2016,29/03/2017,1,1503,269.0,0,1,4,0
1373,2,3,23/04/2014,21/02/2018,1,1585,5508.0,0,4,5,0
3438,2,3,19/11/2015,24/01/2017,1,1503,3689.0,0,3,8,0
3036,2,3,08/06/2016,16/03/2017,1,1874,3686.0,0,2,3,0
2824,2,2,08/09/2016,06/04/2017,1,1507,4828.0,0,2,7,0
2898,2,2,02/12/2016,24/03/2017,1,1533,1443.0,0,1,1,0
3642,2,3,18/10/2013,19/12/2016,1,1542,1541.0,0,4,7,0
